# Execution of SoS workflow

The content of this chapter is largely applicable to batch mode but we represent it in Jupyter notebook for easy reproduction of results. If you are not familiar with Jupyter notebook, please refer to chapter [SoS Kernel](SoS_Kernel.html) for details. For the impatience, the magics

* `%sandgox` execute the cell in a temporary directory
* `!cmd` execute shell command `cmd`
* `%run` run the cell as if in command line with specified options
* `%set` set options that would be included in each `$run` command

## logging level

SoS uses a logging system to output all sorts of information during the execution of workflows. The amount of output can be controlled by logging level, which can be `error` (0), `warning` (1), `info` (2), `debug` (3), and `trace` (4). The default logging level for SoS is `info` in batch mode and `warning` in interactive mode.

For example, logging at `info` level would produce message indicating the steps executed and input output files, but nothing but warning and errors at the `warning` level.

In [2]:
%sandbox
!touch a.txt

%run -v2
[10]
[20]
input: 'a.txt'
[30]
[40]

INFO: Running default_10: 
INFO: Running default_20: 
INFO: input:    ['a.txt']
INFO: Running default_30: 
INFO: Running default_40: 


In [3]:
%sandbox
!touch a.txt

%run -v1
[10]
[20]
input: 'a.txt'
[30]
[40]

## `dryrun` mode

The `dryrun` mode is used to check for syntax errors of a SoS script without actually executing any of the actions. It can be specified with option `-n`. For example, running the following script in dryrun mode would produce an error message

In [4]:
%run -n
[10, skip=False]
sh:
   echo "I am command echo"

File contains parsing errors: <string>
	[line  2]: [10, skip=20]
Invalid statements: SyntaxError('invalid syntax', ('<string>', 1, 10, '[10, skip=20]\n'))

Of course the action `sh` would not be executed even after the script is fixed

In [5]:
%run -n
[10: skip=False]
sh:
   echo "I am command echo"

## Runtime signature

SoS uses <font color='red'>runtime signatures</font> to keep track of <font color='red'>execution units</font>, namely the input, output, and dependent targets, and related SoS variables of a piece of workflow. SoS tracks execution of statements at the step level for each [input group](../documentation/SoS_Step.html) and saves runtime signatures at a folder called `.sos` under the project directory. The runtime signatures are used to

1. Avoid repeated execution of identical units, and
2. Keep track of workflow related files for project management

The second usage would be described in detail in [Project Management](Project_Management.html).

SoS workflows can be executed in batch mode and in interactive mode using the SoS kernel in Jupyter notebook or qtconsole. Because the SoS kernel is mostly used to execute short statements in SOS and other kernels, runtime signatures are by default set to `ignore` in interactive mode (and to `default` in batch mode). To demonstrate these features in Jupyter notebook, we will need to first turn it on, using option

In [ ]:
%set -s default

### Re-execution of executed steps

## Change system $PATH

There are cases where you would like to use a specific version of programs for your workflow but do not want to change the system `$PATH` because of its global effect. In this case you can prepend pathes to these executables to `$PATH` using option `-b`.

The following example first cretes a executable `ls` in `tmp` with an `echo` command. Using the option `-b tmp`, the `tmp` directory is prefixed to the system `$PATH` before the workflow is executed. The consequence is that this fake `ls` supersedes the system `ls` when `ls` is called in `step_10` of the workflow.

In [4]:
%sandbox
!mkdir tmp
!echo "#!/bin/bash" > tmp/ls
!echo "echo This is fake ls" >> tmp/ls
!chmod +x tmp/ls

%run -b tmp
[10]
sh:
    ls

This is fake ls


The `-b` option has a default value `~/.sos/bin`, so any command under `~/.sos/bin` would be executed (before system command with the same name) even if the executables are not under system `$PATH`. This feature allows you to create commands that would only be used inside SoS-scripts, and more interestingly, allows you to create executable or install programs on-the-fly.

For example, step 20 of the following workflow depends on an executable `lls` that is not a system executable.

In [3]:
%sandbox
!rm -f ~/.sos/bin/lls

[install_lls: provides=executable('lls')]
run:
    echo "#!/bin/bash" > ~/.sos/bin/lls
    echo "echo This is lls" >> ~/.sos/bin/lls
    chmod +x ~/.sos/bin/lls

[20]
depends: executable('lls')
run:
    lls

This is lls


because `lls` is created under `~/.sos/bin`, it would be immediately available to SoS after the `install_lls` step. This works for any program as long as you can create a symbolic link under `~/.sos/bin` after its installation.

You can always disable this behavior by setting option `-b` without value.